In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from os import listdir
from os.path import isfile, join

In [2]:
data_false=pd.read_excel(r'C:\Users\kerie\Scripts\Машинное обучение\Системы К\Отказы ТЛМ\Отказы ТЛМ дата создания 2022 год.xlsx')

In [3]:
data_false=data_false.rename(columns={"Статус ТЛМ задания":"Статус_ТЛМ"})
data_false=data_false.query('ВыручкаОтПродаж >= 0 and Статус_ТЛМ == "Отказ"')

In [4]:
data_false.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64792 entries, 0 to 68562
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Контрагент              64792 non-null  object        
 1   Код                     64792 non-null  int64         
 2   ИНН                     64792 non-null  int64         
 3   Наименование            64792 non-null  object        
 4   ВидДеятельности         64792 non-null  object        
 5   СистемаНалогообложения  64792 non-null  object        
 6   ВыручкаОтПродаж         64792 non-null  int64         
 7   КодОКВЭД                12419 non-null  object        
 8   Адрес                   23905 non-null  object        
 9   РегионОбслуживания      64792 non-null  object        
 10  Статус                  64792 non-null  object        
 11  НаправлениеБизнеса      64792 non-null  object        
 12  ДатаСозданияТЛМ         64792 non-null  dateti

- 64792 ТЛМ заданий с результатом ОТКАЗ, по нарпавлению бизнеса "1.1 Реализ. систем К+";
- Данные без пустых значений в столбцах СистемаНалогообложения, ВыручкаОтПродаж.

In [5]:
mypath=r'C:\Users\kerie\Scripts\Машинное обучение\Системы К\Сделки'
names = [f for f in listdir(mypath) if isfile(join(mypath, f))]
len(names)
names

['2018.xlsx', '2019.xlsx', '2020.xlsx', '2021.xlsx', '2022.xlsx']

In [6]:
excl_list=[]
for i in names:
    file=f'{mypath}\{i}'
    data=pd.read_excel(file)
     # Год
    Year=i[:i.index('.')]
    data["Год_сделки"] = int(Year)
    excl_list.append(data)

In [7]:
data_true = pd.concat(excl_list)
data_true.shape

(6978, 19)

In [8]:
# Оставим данные, где только есть выручка компании и система налообложения известна
data_true=data_true.loc[((data_true['ВыручкаОтПродаж']>=0) & (~data_true['СистемаНалогообложения'].isnull()))] 
data_true.shape

(3175, 19)

In [9]:
data_true.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3175 entries, 0 to 1404
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Ссылка                  3175 non-null   object        
 1   ПометкаУдаления         3175 non-null   object        
 2   Код                     3175 non-null   int64         
 3   ИНН                     3175 non-null   object        
 4   Наименование            3175 non-null   object        
 5   ВидДеятельности         979 non-null    object        
 6   ЮрФизЛицо               3175 non-null   object        
 7   СистемаНалогообложения  3175 non-null   object        
 8   ВыручкаОтПродаж         3175 non-null   float64       
 9   КодОКВЭД                628 non-null    object        
 10  Адрес                   3172 non-null   object        
 11  РегионОбслуживания      3175 non-null   object        
 12  Статус                  3175 non-null   object  

3175 сделок, по нарпавлению бизнеса "1.1 Реализ. систем К+";
Данные без пустых значений в столбцах СистемаНалогообложения, ВыручкаОтПродаж.

# Для обучения моделей будем использовать следующие параметры:

- СистемаНалогообложения;
- ВыручкаОтПродаж;
- РегионОбслуживания;
- КодОКВЭД_69.

# Кластеризируем данные со статусом ТЛМ задания "ОТКАЗ", для создания групп

Для кластеризации будем использовать метод, DBSCAN.

Для информации:

В данном примере мы используем метод DBSCAN для кластеризации исходного датасета. Параметр eps задает максимальное расстояние между соседними точками, чтобы они считались одним кластером. Параметр min_samples определяет минимальное количество соседних точек, необходимое для формирования кластера.

Обратите внимание, что DBSCAN может определить количество кластеров автоматически, исходя из структуры данных.

In [10]:
data_false=data_false[['СистемаНалогообложения','ВыручкаОтПродаж','РегионОбслуживания','Статус','КодОКВЭД_69']]
data_false.head()

,СистемаНалогообложения,ВыручкаОтПродаж,РегионОбслуживания,Статус,КодОКВЭД_69
0,ОСНО,4147256000,Москва\МО,Действующее,Нет
2,УСНО 15%,85053000,Москва\МО,Действующее,Нет
4,УСНО 6%,129228000,Москва\МО,Действующее,Нет
5,ОСНО,26793000,Москва\МО,Действующее,Нет
6,УСНО 6%,66092000,Москва\МО,Действующее,Нет


In [11]:
#Номинативные переменные, меняем на числовые
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

category=['СистемаНалогообложения','РегионОбслуживания','Статус','КодОКВЭД_69']

for i in category:
    le.fit(data_false[i])
    data_false[i+'_int']=le.transform(data_false[i])

data_false.head()

,СистемаНалогообложения,ВыручкаОтПродаж,РегионОбслуживания,Статус,КодОКВЭД_69,СистемаНалогообложения_int,РегионОбслуживания_int,Статус_int,КодОКВЭД_69_int
0,ОСНО,4147256000,Москва\МО,Действующее,Нет,4,1,0,1
2,УСНО 15%,85053000,Москва\МО,Действующее,Нет,7,1,0,1
4,УСНО 6%,129228000,Москва\МО,Действующее,Нет,8,1,0,1
5,ОСНО,26793000,Москва\МО,Действующее,Нет,4,1,0,1
6,УСНО 6%,66092000,Москва\МО,Действующее,Нет,8,1,0,1


# Стандартизация данных

In [12]:
from sklearn.preprocessing import StandardScaler
x=data_false[['ВыручкаОтПродаж','СистемаНалогообложения_int','РегионОбслуживания_int','Статус_int','КодОКВЭД_69_int']]
x=np.nan_to_num(x)
scaler= StandardScaler().fit_transform(x) 

# Кластеризируем данные

In [13]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.3, min_samples=5)
dbscan.fit(scaler)

labels = dbscan.labels_
data_false['kluster']=labels
print(max(labels)+1,'')

51 


In [14]:
# Сохраняю отказы, разбитые по кластерам в файлик
data_false.to_excel("кластеризация_отказов.xlsx")
#data_false[data_false['kluster']==-1].count()

Выводы:
- Метод DBSCAN разбил выборку на 51 кластер;
- Значения кластера с меткой -1, это выброс (не стандартные значения) таких получилось 108.

# Для обучения модели, отберем значения "Отказов" из каждого кластера, чтобы получилось 3125 значений, и 3125 значений сделок.

In [15]:
data_false[data_false['kluster']==16][:]

,СистемаНалогообложения,ВыручкаОтПродаж,РегионОбслуживания,Статус,КодОКВЭД_69,СистемаНалогообложения_int,РегионОбслуживания_int,Статус_int,КодОКВЭД_69_int,kluster
162,УСНО 15%,0,Москва\МО,Недействующие,Нет,7,1,1,1,16
1693,УСНО 15%,706000,Москва\МО,Недействующие,Нет,7,1,1,1,16
2202,УСНО 15%,0,Москва\МО,Недействующие,Нет,7,1,1,1,16
4125,УСНО 15%,69158000,Москва\МО,Недействующие,Нет,7,1,1,1,16
4531,УСНО 15%,205125000,Москва\МО,Недействующие,Нет,7,1,1,1,16
...,...,...,...,...,...,...,...,...,...,...
63745,УСНО 15%,1757000,Москва\МО,Недействующие,Нет,7,1,1,1,16
64896,УСНО 15%,54219000,Москва\МО,Недействующие,Нет,7,1,1,1,16
66616,УСНО 15%,93041000,Москва\МО,Недействующие,Нет,7,1,1,1,16
67724,УСНО 15%,28561000,Москва\МО,Недействующие,Нет,7,1,1,1,16


In [16]:
data_f=pd.concat([
                  data_false[data_false['kluster']==50][1:],
                  data_false[data_false['kluster']==49][1:],
                  data_false[data_false['kluster']==48][1:],
                  data_false[data_false['kluster']==47][1:], 
                  data_false[data_false['kluster']==46][1:],
                  data_false[data_false['kluster']==45][1:],
                  data_false[data_false['kluster']==44][1:],
                  data_false[data_false['kluster']==43][:1],
                  data_false[data_false['kluster']==42][:5],
                  data_false[data_false['kluster']==41][1:],
                  data_false[data_false['kluster']==40][1:],
                  data_false[data_false['kluster']==39][:1],
                  data_false[data_false['kluster']==38][1:],
                  data_false[data_false['kluster']==37][:13],
                  data_false[data_false['kluster']==36][1:],
                  data_false[data_false['kluster']==35][:22],
                  data_false[data_false['kluster']==34][1:],
                  data_false[data_false['kluster']==33][1:],
                  data_false[data_false['kluster']==32][1:],
                  data_false[data_false['kluster']==31][1:],
                  data_false[data_false['kluster']==30][1:],
                  data_false[data_false['kluster']==29][1:],
                  data_false[data_false['kluster']==28][1:],
                  data_false[data_false['kluster']==27][1:], 
                  data_false[data_false['kluster']==26][1:],
                  data_false[data_false['kluster']==25][1:],
                  data_false[data_false['kluster']==24].sample(n=109),
                  data_false[data_false['kluster']==23][:34],
                  data_false[data_false['kluster']==22].sample(n=109),
                  data_false[data_false['kluster']==21][1:],
                  data_false[data_false['kluster']==20][1:],
                  data_false[data_false['kluster']==19][1:],
                  data_false[data_false['kluster']==18][1:],
                  data_false[data_false['kluster']==17].sample(n=109),
                  data_false[data_false['kluster']==16][:115],
                  data_false[data_false['kluster']==15].sample(n=109),
                  data_false[data_false['kluster']==14].sample(n=109),
                  data_false[data_false['kluster']==13].sample(n=109),
                  data_false[data_false['kluster']==12].sample(n=109),
                  data_false[data_false['kluster']==11].sample(n=109),
                  data_false[data_false['kluster']==10].sample(n=109),
                  data_false[data_false['kluster']==9].sample(n=109),
                  data_false[data_false['kluster']==8].sample(n=119),
                  data_false[data_false['kluster']==7][1:],
                  data_false[data_false['kluster']==6][1:],
                  data_false[data_false['kluster']==5].sample(n=119),
                  data_false[data_false['kluster']==4].sample(n=119),
                  data_false[data_false['kluster']==3].sample(n=119),
                  data_false[data_false['kluster']==2].sample(n=119),
                  data_false[data_false['kluster']==1].sample(n=119),
                  data_false[data_false['kluster']==0].sample(n=119)])


# Отказы
data_f['Сделка']=0
data_f=data_f[['СистемаНалогообложения','ВыручкаОтПродаж','РегионОбслуживания','Статус','КодОКВЭД_69','Сделка']]

# Сделки
data_t=data_true[:3125]. \
rename(columns={"Выручка \nКомпании, руб":"ВыручкаОтПродаж"})
data_t['Сделка']=1
data_t=data_t[['СистемаНалогообложения','ВыручкаОтПродаж','РегионОбслуживания','Статус','КодОКВЭД_69','Сделка']]

data=pd.concat([data_f,data_t])

#Номинативные переменные, меняем на числовые
le=LabelEncoder()
category=['СистемаНалогообложения','РегионОбслуживания','Статус','КодОКВЭД_69']

for i in category:
    le.fit(data[i])
    data[i+'_int']=le.transform(data[i])

# перемешаем Датасет
data = data.sample(frac=1)
data

,СистемаНалогообложения,ВыручкаОтПродаж,РегионОбслуживания,Статус,КодОКВЭД_69,Сделка,СистемаНалогообложения_int,РегионОбслуживания_int,Статус_int,КодОКВЭД_69_int
747,ОСНО,8883000.0,Москва\МО,Действующее,Да,1,4,1,0,0
150,ОСНО,154137000.0,Москва\МО,Действующее,Нет,1,4,1,0,1
48090,ЕНВД,131339000.0,Москва\МО,Действующее,Нет,0,2,1,0,1
476,ОСНО,204926000.0,Москва\МО,Действующее,Нет,1,4,1,0,1
298,УСНО 6%,132519000.0,Москва\МО,Недействующие,Нет,1,8,1,1,1
...,...,...,...,...,...,...,...,...,...,...
650,УСНО 15%,20184000.0,Москва\МО,Действующее,Нет,1,7,1,0,1
872,ОСНО,124311000.0,Москва\МО,Действующее,Нет,1,4,1,0,1
1061,ОСНО,490773000.0,Москва\МО,Действующее,Да,1,4,1,0,0
709,ОСНО,41684000.0,Москва\МО,Действующее,Нет,1,4,1,0,1


# Данные для прогноза

In [17]:
data_f_p=pd.concat([
                    data_false[data_false['kluster']==50][:1],
                    data_false[data_false['kluster']==49][:1],
                    data_false[data_false['kluster']==48][:1],
                    data_false[data_false['kluster']==47][:1],
                    data_false[data_false['kluster']==46][:1],
                    data_false[data_false['kluster']==45][:1],
                    data_false[data_false['kluster']==44][1:],
                    data_false[data_false['kluster']==43][1:2],
                    data_false[data_false['kluster']==42][5:],
#                     data_false[data_false['kluster']==41][:1],
#                     data_false[data_false['kluster']==40][:1],
#                     data_false[data_false['kluster']==39][1:2],
#                     data_false[data_false['kluster']==38][:1],
#                     data_false[data_false['kluster']==37][13:],
                    data_false[data_false['kluster']==36][:1],
                    data_false[data_false['kluster']==35][22:],
                    data_false[data_false['kluster']==34][:1],
                    data_false[data_false['kluster']==33][:1],
                    data_false[data_false['kluster']==32][:1],
                    data_false[data_false['kluster']==31][:1],
                    data_false[data_false['kluster']==30][:1], # остановился тут 
                    data_false[data_false['kluster']==29][:1],
                    data_false[data_false['kluster']==28][:1],
                    data_false[data_false['kluster']==27][:1],
                    data_false[data_false['kluster']==26][:1],
                    data_false[data_false['kluster']==25][:1],
                    data_false[data_false['kluster']==24].sample(n=1),
                    data_false[data_false['kluster']==23][34:],
                    data_false[data_false['kluster']==22].sample(n=1),
                    data_false[data_false['kluster']==21][:1],
                    data_false[data_false['kluster']==20][:1],
                    data_false[data_false['kluster']==19][:1],
                    data_false[data_false['kluster']==18][:1],
                    data_false[data_false['kluster']==17].sample(n=1),
                    data_false[data_false['kluster']==16][115:],
                    data_false[data_false['kluster']==15].sample(n=1),
                    data_false[data_false['kluster']==14].sample(n=1),
                    data_false[data_false['kluster']==13].sample(n=1),
                    data_false[data_false['kluster']==12].sample(n=1),
                    data_false[data_false['kluster']==11].sample(n=1),
                    data_false[data_false['kluster']==10].sample(n=1),
                    data_false[data_false['kluster']==9].sample(n=1),
                    data_false[data_false['kluster']==8].sample(n=1),
                    data_false[data_false['kluster']==7][:1],
                    data_false[data_false['kluster']==6][:1],
                    data_false[data_false['kluster']==5].sample(n=1),
                    data_false[data_false['kluster']==4].sample(n=1),
                    data_false[data_false['kluster']==3].sample(n=1),
                    data_false[data_false['kluster']==2].sample(n=1),
                    data_false[data_false['kluster']==1].sample(n=1),
                    data_false[data_false['kluster']==0].sample(n=1)])
 
data_f_p['Сделка']=0
data_f_p=data_f_p[['СистемаНалогообложения','ВыручкаОтПродаж','РегионОбслуживания','Статус','КодОКВЭД_69','Сделка']]

data_t_p=data_true[3125:]

data_t_p['Сделка']=1
data_t_p=data_t_p[['СистемаНалогообложения','ВыручкаОтПродаж','РегионОбслуживания','Статус','КодОКВЭД_69','Сделка']]

data_p=pd.concat([data_t_p,data_f_p])

# data_p=data_p[['СистемаНалогообложения','ВыручкаОтПродаж','РегионОбслуживания','Статус','КодОКВЭД_69','Сделка']]

data_p.to_excel("Данные для прогноза.xlsx",index=False)
data_p.shape

C:\Users\kerie\AppData\Local\Temp\ipykernel_20500\765468338.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_t_p['Сделка']=1


(100, 6)

In [18]:
# Номинативные переменные, заменяем на качественные, данными, как при обучении
sn=['Аутсорсинг','Бюджет','ЕНВД','ЕСХН','ОСНО','ОСНО,УСНО','Патент','УСНО 15%','УСНО 6%']
def alert(row):
    for i in range(0,9):
         if row['СистемаНалогообложения']==sn[i]:
                return i
data_p['СистемаНалогообложения']=data_p.apply(alert, axis=1) 

ro=['Другой регион','Москва\МО']
def alert_r(row):
    for i in range(0,2):
         if row['РегионОбслуживания']==ro[i]:
                return i
data_p['РегионОбслуживания']=data_p.apply(alert_r, axis=1)

st=['Действующее','Недействующие','Нет информации']
def alert_s(row):
    for i in range(0,3):
         if row['Статус']==st[i]:
                return i
data_p['Статус']=data_p.apply(alert_s, axis=1)

kod=['Да','Нет']
def alert_с(row):
    for i in range(0,2):
         if row['КодОКВЭД_69']==kod[i]:
                return i
data_p['КодОКВЭД_69']=data_p.apply(alert_с, axis=1)

data_p = data_p.sample(frac=1)

In [19]:
data_p.rename(columns={"СистемаНалогообложения":"СистемаНалогообложения_int",
                              "РегионОбслуживания":"РегионОбслуживания_int",
                              "КодОКВЭД_69":"КодОКВЭД_69_int",
                              "Статус":"Статус_int"}, inplace = True) 
# перемешаем Датасет
data_p.head(10)

,СистемаНалогообложения_int,ВыручкаОтПродаж,РегионОбслуживания_int,Статус_int,КодОКВЭД_69_int,Сделка
911,4,2.601869e+11,1,0,1,0
55150,4,6.865930e+08,1,0,0,0
615,2,1.693710e+08,0,0,1,0
1390,4,1.097200e+07,1,0,1,1
1346,4,1.287470e+08,1,0,1,1
1037,1,7.015000e+06,1,0,1,0
1351,5,1.847332e+09,1,0,1,1
1362,7,1.054000e+07,1,0,0,1
318,4,2.093156e+11,1,0,1,0
1391,4,3.559040e+08,1,0,1,1


# Модель на CatBoost

In [20]:
from catboost import CatBoostClassifier
from catboost import Pool
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [21]:
data.shape

(6250, 10)

In [22]:
#Отбор признаков
# Будем перебирать все комбинации атрибутов
from itertools import combinations
name_columns=list(data[['СистемаНалогообложения_int','ВыручкаОтПродаж',
                        'РегионОбслуживания_int','Статус_int','КодОКВЭД_69_int']].columns)
feature = sum([list(map(list, combinations(name_columns, i))) for i in range(len(name_columns) + 1)], [])
feature.pop(0)
feature

[['СистемаНалогообложения_int'],
 ['ВыручкаОтПродаж'],
 ['РегионОбслуживания_int'],
 ['Статус_int'],
 ['КодОКВЭД_69_int'],
 ['СистемаНалогообложения_int', 'ВыручкаОтПродаж'],
 ['СистемаНалогообложения_int', 'РегионОбслуживания_int'],
 ['СистемаНалогообложения_int', 'Статус_int'],
 ['СистемаНалогообложения_int', 'КодОКВЭД_69_int'],
 ['ВыручкаОтПродаж', 'РегионОбслуживания_int'],
 ['ВыручкаОтПродаж', 'Статус_int'],
 ['ВыручкаОтПродаж', 'КодОКВЭД_69_int'],
 ['РегионОбслуживания_int', 'Статус_int'],
 ['РегионОбслуживания_int', 'КодОКВЭД_69_int'],
 ['Статус_int', 'КодОКВЭД_69_int'],
 ['СистемаНалогообложения_int', 'ВыручкаОтПродаж', 'РегионОбслуживания_int'],
 ['СистемаНалогообложения_int', 'ВыручкаОтПродаж', 'Статус_int'],
 ['СистемаНалогообложения_int', 'ВыручкаОтПродаж', 'КодОКВЭД_69_int'],
 ['СистемаНалогообложения_int', 'РегионОбслуживания_int', 'Статус_int'],
 ['СистемаНалогообложения_int', 'РегионОбслуживания_int', 'КодОКВЭД_69_int'],
 ['СистемаНалогообложения_int', 'Статус_int', 'Ко

In [25]:
x_pred=data_p.iloc[:,:-1]
y_true=data_p.iloc[:,-1]

In [131]:
# Данные
x=data[['СистемаНалогообложения_int','ВыручкаОтПродаж',
        'РегионОбслуживания_int','Статус_int','КодОКВЭД_69_int']] # Зависимые переменные 
y=data['Сделка'] # Прогноз

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20,random_state=42)


info_test=[]
info_p=[]

# будем перебирать все признаки 
# и смотреть метрику на тестовой выборке,
# и на выборке, которая не учавствует в обучении.
# Метрики для оценки качества Accuracy и ROK-AUC


for i in feature:
    train_pool=Pool(data=x_train[i], label=y_train)
    validation_pool=Pool(data=x_test[i],label=y_test)

    model = CatBoostClassifier(
    iterations=11500,
    learning_rate = 0.003,
    random_seed=63,
    custom_loss=['AUC', 'Accuracy'],
    use_best_model=True)
    
    
    model.fit(
    train_pool, 
    eval_set=validation_pool,
    verbose=False,
    plot=False);
    
    pred=model.predict(x_pred[i])
    pred_test=model.predict(x_test[i])
    
    info_p.append([i,round(metrics.accuracy_score(y_true,pred),2), round(metrics.roc_auc_score(y_true,pred),2)])
    info_test.append([i,round(metrics.accuracy_score(y_test,pred_test),2), round(metrics.roc_auc_score(y_test,pred_test),2)]) 

In [133]:
print('Результат на тестовой выборке, Accuracy and ROK-AUC')
info_test = sorted(info_test, key=lambda x: (x[1], -x[1]))
for i in info_test:
    print(i)

Результат на тестовой выборке, Accuracy and ROK-AUC
[['ВыручкаОтПродаж'], 0.56, 0.56]
[['Статус_int'], 0.59, 0.59]
[['ВыручкаОтПродаж', 'Статус_int'], 0.6, 0.61]
[['КодОКВЭД_69_int'], 0.61, 0.61]
[['ВыручкаОтПродаж', 'КодОКВЭД_69_int'], 0.61, 0.61]
[['РегионОбслуживания_int'], 0.65, 0.65]
[['ВыручкаОтПродаж', 'РегионОбслуживания_int'], 0.66, 0.67]
[['Статус_int', 'КодОКВЭД_69_int'], 0.67, 0.68]
[['ВыручкаОтПродаж', 'Статус_int', 'КодОКВЭД_69_int'], 0.67, 0.68]
[['СистемаНалогообложения_int'], 0.69, 0.69]
[['РегионОбслуживания_int', 'Статус_int'], 0.69, 0.7]
[['СистемаНалогообложения_int', 'ВыручкаОтПродаж'], 0.7, 0.7]
[['СистемаНалогообложения_int', 'КодОКВЭД_69_int'], 0.71, 0.71]
[['РегионОбслуживания_int', 'КодОКВЭД_69_int'], 0.71, 0.71]
[['ВыручкаОтПродаж', 'РегионОбслуживания_int', 'Статус_int'], 0.71, 0.72]
[['ВыручкаОтПродаж', 'РегионОбслуживания_int', 'КодОКВЭД_69_int'], 0.72, 0.73]
[['РегионОбслуживания_int', 'Статус_int', 'КодОКВЭД_69_int'], 0.74, 0.74]
[['СистемаНалогообложен

In [134]:
print('Результат на прогнозной выборке, Accuracy and ROK-AUC')
info_p = sorted(info_p, key=lambda x: (x[1], -x[1]))
for i in info_p:
    print(i)

Результат на прогнозной выборке, Accuracy and ROK-AUC
[['ВыручкаОтПродаж'], 0.56, 0.56]
[['КодОКВЭД_69_int'], 0.6, 0.6]
[['СистемаНалогообложения_int'], 0.63, 0.63]
[['Статус_int'], 0.63, 0.63]
[['СистемаНалогообложения_int', 'КодОКВЭД_69_int'], 0.64, 0.64]
[['Статус_int', 'КодОКВЭД_69_int'], 0.68, 0.68]
[['СистемаНалогообложения_int', 'РегионОбслуживания_int'], 0.69, 0.69]
[['СистемаНалогообложения_int', 'Статус_int'], 0.69, 0.69]
[['РегионОбслуживания_int'], 0.7, 0.7]
[['СистемаНалогообложения_int', 'ВыручкаОтПродаж'], 0.7, 0.7]
[['ВыручкаОтПродаж', 'Статус_int'], 0.72, 0.72]
[['ВыручкаОтПродаж', 'КодОКВЭД_69_int'], 0.72, 0.72]
[['РегионОбслуживания_int', 'КодОКВЭД_69_int'], 0.73, 0.73]
[['СистемаНалогообложения_int', 'ВыручкаОтПродаж', 'КодОКВЭД_69_int'], 0.73, 0.73]
[['СистемаНалогообложения_int', 'Статус_int', 'КодОКВЭД_69_int'], 0.74, 0.74]
[['СистемаНалогообложения_int', 'ВыручкаОтПродаж', 'Статус_int'], 0.75, 0.75]
[['ВыручкаОтПродаж', 'РегионОбслуживания_int'], 0.76, 0.76]
[['

In [135]:
# Оценка влияния фичей (признаков) в % соотношении
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,РегионОбслуживания_int,53.746889
1,СистемаНалогообложения_int,27.512867
2,ВыручкаОтПродаж,7.531443
3,Статус_int,6.941058
4,КодОКВЭД_69_int,4.267743


In [136]:
# Оценка влияния фичей (признаков) на loss функцию в % соотношении
model.get_feature_importance(
    validation_pool,   #LossFunctionChange
    prettified=True)

,Feature Id,Importances
0,РегионОбслуживания_int,52.590809
1,СистемаНалогообложения_int,27.699921
2,ВыручкаОтПродаж,7.769473
3,Статус_int,7.158769
4,КодОКВЭД_69_int,4.781028


In [138]:
print('Результат на прогнозной выборке, Accuracy and ROK-AUC')
print(info_p[-2])
print()
print('Результат на тестовой выборке, Accuracy and ROK-AUC')
print(info_test[-1])

Результат на прогнозной выборке, Accuracy and ROK-AUC
[['СистемаНалогообложения_int', 'ВыручкаОтПродаж', 'РегионОбслуживания_int', 'Статус_int', 'КодОКВЭД_69_int'], 0.88, 0.88]

Результат на тестовой выборке, Accuracy and ROK-AUC
[['СистемаНалогообложения_int', 'ВыручкаОтПродаж', 'РегионОбслуживания_int', 'Статус_int', 'КодОКВЭД_69_int'], 0.83, 0.83]


# Итоговая модель CatBoost

In [26]:
# Данные
x=data[['СистемаНалогообложения_int','ВыручкаОтПродаж',
        'РегионОбслуживания_int','Статус_int','КодОКВЭД_69_int']] # Зависимые переменные 
y=data['Сделка'] # Прогноз

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15,random_state=42)

In [27]:
train_pool=Pool(data=x_train, label=y_train)
validation_pool=Pool(data=x_test,label=y_test)

info_p_new=[]
info_test_new=[]


model_over = CatBoostClassifier(
             iterations=11500,
             learning_rate = 0.003,
             random_seed=63,
             custom_loss=['AUC', 'Accuracy'],
             use_best_model=True)

    
model_over.fit(
           train_pool, 
           eval_set=validation_pool,
           verbose=False,
           plot=True);
    
pred=model_over.predict(x_pred)
pred_test=model_over.predict(x_test)
    
info_p_new.append([round(metrics.accuracy_score(y_true,pred),2), round(metrics.roc_auc_score(y_true,pred),2)])
info_test_new.append([round(metrics.accuracy_score(y_test,pred_test),2), round(metrics.roc_auc_score(y_test,pred_test),2)]) 

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [28]:
print('Результат на тестовой, Accuracy and ROK-AUC')
print(info_test_new)

Результат на тестовой, Accuracy and ROK-AUC
[[0.86, 0.86]]


In [29]:
print('Результат на прогнозной выборке, Accuracy and ROK-AUC')
print(info_p_new)

Результат на прогнозной выборке, Accuracy and ROK-AUC
[[0.88, 0.88]]


In [30]:
#model_over.save_model("Catboost_Model_K+.bin")

# Модель Случайный лес RandomForest

In [32]:
from sklearn.ensemble import RandomForestClassifier
# Объявляем и обучаем модель 
rfc = RandomForestClassifier().fit(x_train, y_train)

In [34]:
pred_rfc=rfc.predict(x_pred)
pred_test_rfc=rfc.predict(x_test)

print(round(metrics.accuracy_score(y_true,pred_rfc),2), round(metrics.roc_auc_score(y_true,pred_rfc),2))
print(round(metrics.accuracy_score(y_test,pred_test_rfc),2), round(metrics.roc_auc_score(y_test,pred_test_rfc),2)) 

0.79 0.79
0.8 0.8


In [41]:
import pickle
# Сохранение модели в файл
with open('RFC_Model_K.pkl', 'wb') as file:
    pickle.dump(rfc, file)